In [1]:
import cv2, os, numpy as np, pandas as pd
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, RandomFlip, RandomRotation, RandomZoom, RandomContrast, RandomBrightness, Dropout
from keras.callbacks import EarlyStopping
from keras.models import load_model
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [2]:
def img_process(img):
      
      img1 = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
      img2 = cv2.resize(img1, (150, 150)).reshape(1, 150, 150, 3)/255.0
      return img2

In [ ]:
path = r'C:\LUMINAR\PROJECT\SmartGrocery\train'
folders = os.listdir(path)
print(folders), print(len(folders))
X = []
Y = []
for item_folder in folders:
      item_folder_path = os.path.join(path, item_folder)
      # print(fruit_name_path)
      item_images_files = os.listdir(item_folder_path)
      # print(item_images)
      for item_image_file in item_images_files:
            item_image_path = os.path.join(item_folder_path, item_image_file)
            img = cv2.imread(item_image_path)
            if img is None:
                  continue
            img1 = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
            img2 = cv2.resize(img1, (150, 150)) / 255.0
            X.append(img2)
            Y.append(folders.index(item_folder))
            print(item_folder)
            print(folders.index(item_folder))
X = np.array(X, dtype=np.float32)
Y = np.array(Y)
X.shape, Y.shape

In [ ]:
path = r'C:\LUMINAR\PROJECT\SmartGrocery\test'
folders = os.listdir(path)
print(folders), print(len(folders))
X_test = []
Y_test = []
for item_folder in folders:
      item_folder_path = os.path.join(path, item_folder)
      # print(fruit_name_path)
      item_images_files = os.listdir(item_folder_path)
      # print(item_images)
      for item_image_file in item_images_files:
            item_image_path = os.path.join(item_folder_path, item_image_file)
            img = cv2.imread(item_image_path)
            if img is None:
                  continue
            img1 = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
            img2 = cv2.resize(img1, (150, 150)) / 255.0
            X_test.append(img2)
            Y_test.append(folders.index(item_folder))
            print(item_folder)
            print(folders.index(item_folder))
X_test = np.array(X_test, dtype=np.float32)
Y_test = np.array(Y_test)
X_test.shape, Y_test.shape

In [ ]:
path = r'C:\LUMINAR\PROJECT\SmartGrocery\validation'
folders = os.listdir(path)
print(folders), print(len(folders))
X_vali = []
Y_vali = []
for item_folder in folders:
      item_folder_path = os.path.join(path, item_folder)
      # print(fruit_name_path)
      item_images_files = os.listdir(item_folder_path)
      # print(item_images)
      for item_image_file in item_images_files:
            item_image_path = os.path.join(item_folder_path, item_image_file)
            img = cv2.imread(item_image_path)
            if img is None:
                  continue
            img1 = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
            img2 = cv2.resize(img1, (150, 150)) / 255.0
            X_vali.append(img2)
            Y_vali.append(folders.index(item_folder))
            print(item_folder)
            print(folders.index(item_folder))
X_vali = np.array(X_vali, dtype=np.float32)
Y_vali = np.array(Y_vali)
X_vali.shape, Y_vali.shape

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
import tensorflow as tf

# Data Augmentation
data_augmentation = Sequential([
    tf.keras.layers.RandomFlip("horizontal_and_vertical"),  # Randomly flip images
    tf.keras.layers.RandomRotation(0.2),  # Randomly rotate images
    tf.keras.layers.RandomZoom(0.2),  # Random zoom
    tf.keras.layers.RandomTranslation(0.2, 0.2)  # Random translation
])

# Model Definition
model = Sequential()

# Add Data Augmentation Layer before Conv Layers
model.add(data_augmentation)

# Add Convolutional Layers
model.add(Conv2D(32, (3, 3), input_shape=(150, 150, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Conv2D(128, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

# Add Flatten Layer
model.add(Flatten())

# Fully Connected Layers
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(36, activation='softmax'))  # Output layer

# Compile Model
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])




# Train the Model
model.fit(
    X,
    Y,
    validation_data=validation_generator,
    epochs=50,
    
    batch_size = 100
)


In [13]:
model.save('my_model2.keras')

In [ ]:
model.evaluate(X_test, Y_test)

In [15]:
model = load_model(r'C:\LUMINAR\PROJECT\SmartGrocery\my_model2.keras')

In [ ]:
path = r'C:\LUMINAR\PROJECT\SmartGrocery\test\kiwi\Image_3.jpg'
image = cv2.imread(path)
g = model.predict(img_process(image))
folders[g.argmax()]

In [ ]:
import cv2

# Load the video capture
video = cv2.VideoCapture(0)  # Use 0 for the default camera or provide a video file path

# Ensure the video capture is open
if not video.isOpened():
    print("Error: Unable to access the camera or video.")
    exit()

# Main loop to process the video feed
while True:
    ret, image = video.read()  # Correct variable naming for success flag and frame
    if not ret:
        print("Error: Failed to read from video.")
        break

    try:
        # Convert the frame to RGB
        img1 = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

        # Resize and normalize the image
        img2 = cv2.resize(img1, (150, 150)).reshape(1, 150, 150, 3) / 255.0

        # Predict the class of the frame
        prediction = model.predict(img2).argmax().item()
        print(folders[prediction])  # Replace `folders` with the actual label list

        # Display the frame
        cv2.imshow("image", image)

        # Exit on pressing 'q'
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

    except Exception as e:
        print(f"Error processing frame: {e}")
        break

# Release resources
video.release()
cv2.destroyAllWindows()
